In [2]:
cd ..

/home/astro/phrdhx/automated_exocomet_hunt


In [8]:
import data
import data_old
import loaders

`data.load_bad_times()` loads up the file `tess_bad_times.txt`

In [16]:
test1 = data.load_bad_times()

In [17]:
test2 = data_old.load_bad_times()

---

In [88]:
import pandas as pd
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
from astropy.stats import sigma_clip
import matplotlib.pyplot as plt

In [86]:
def import_XRPlightcurve(file_path):
    """
    Importing the compressed TESS lightcurves from the XRP group.
    file_path: path to file
    :type file_path: pkl
    :returns: 
        - lc - lightcurve data as a DataFrame. 
        - store - Storage of other information (TIC, RA, Dec, TESS magnitude, Camera, Chip)
    """
    data = pd.read_pickle(file_path)

    ## extracting the lightcurve data and converting to Series from lists
    for i in range(len(data)):
        if isinstance(data[i],np.ndarray):
            data[i] = pd.Series(data[i])
    for_df = data[6:] # data[0:6] is not relevant in this case.
    columns = ['time','raw flux','corrected flux','PCA flux','flux error','quality']
    df = pd.DataFrame(data=for_df).T
    df.columns = columns
    return df, data[0:6]


def mad_plots(table,array,median,rms,clip,sector,camera):
    """plots comparisons of MAD at sector camera combination between median, BL's MAD, and statistically clipped MAD"""
    fig,ax = plt.subplots()
    ax.scatter(range(0,len(table)), array, s=2)
    ax.axhline(np.nanmedian(array), c='r',label='median')
    ax.axhline(np.nanmedian(array)+10*np.std(array[900:950]),c='blue',label='visualised MAD') # [900:950] are generally quiet cadences
    ax.axhline(~sig_clip)
    ax.axhline(median + clip*rms, c='orange',label='Sigma Clipped MAD')
    ax.set_xlabel('Cadence Number')
    ax.set_ylabel('Mean Absolute Deviation (MAD)')
    ax.set_title(f'Cadence at {sector}-{camera}')
    ax.legend()
    plt.show()

In [29]:
filename = 'tess_testlcs/tesslcs_sector_5_104_2_min_cadence_targets_tesslc_270577175.pkl'
lc, store = import_XRPlightcurve(filename)

In [31]:
sec = 6
camera = store[4]

In [32]:
mad_df = data.load_mad()

In [37]:
mad_arr = mad_df.loc[:len(lc)-1,f"{sec}-{camera}"]

In [95]:
sig_clip = sigma_clip(mad_arr,sigma=3,masked=True)

In [99]:
mad_arr.values

array([0.00077316, 0.00072744, 0.00057492, 0.00066968, 0.00064916,
              nan, 0.00054146, 0.00059869, 0.00060656, 0.00058808,
       0.00056348, 0.00055266, 0.00054677, 0.00055607, 0.0004952 ,
       0.00053825, 0.00049156, 0.00051544, 0.00045189, 0.00048895,
       0.0004277 , 0.00046132, 0.00045156, 0.00045518, 0.00044403,
       0.00043113, 0.00040401, 0.00040083, 0.00040492, 0.0003766 ,
       0.00040003, 0.00037297,        nan, 0.00037704, 0.00032658,
       0.00034565, 0.00032395, 0.00030803, 0.00031822, 0.00029673,
       0.00027173, 0.0002939 , 0.00034335, 0.00027647, 0.0002897 ,
       0.00028628, 0.00029594, 0.00026248, 0.00027   , 0.00026414,
       0.0002469 , 0.00023948, 0.00026771, 0.00026529, 0.00026053,
       0.00025945, 0.00026337, 0.0002713 , 0.00024187, 0.0002718 ,
       0.0002529 , 0.00027176, 0.00026612, 0.00028353, 0.00028177,
       0.00025969, 0.0002638 , 0.00026538, 0.00025961, 0.00026274,
       0.00026058, 0.00025415, 0.00026985, 0.00026505, 0.00025

In [112]:
mad_cut = mad_arr.values < ~sig_clip.mask

In [110]:
test = mad_arr.values < np.nanmedian(sig_clip) + 3*np.nanstd(sig_clip)
test[test].size

866

In [74]:
mad_cut = ~mad_cut

In [77]:
mad_cut[mad_cut == True].shape

(1085,)

In [84]:
test = mad_arr.values < np.nanmedian(sig_clip) + 3*(np.nanstd(sig_clip))

In [85]:
test[test == True].shape

(866,)

In [113]:
ls

all_snr_gt_5_ok.txt
all_snr_gt_5.txt
analysis_tools_cython.c
analysis_tools_cython.cpython-37m-x86_64-linux-gnu.so*
analysis_tools_cython.pyx
archive_analyse.sh*
artefact_list.txt
batch_analyse.py*
batch_reanalyse_asym.py*
box_sn_gt5_maybeTransit.awk
build/
data/
data_old/
dr2.xml
environment.yml
figs/
figs.ipynb
filters_to_final.awk
kplr003542116-2012088054726_llc.fits
loaders.py
make*
maxsector.py
Notebooks/
output_spoc30min_s6.txt
__pycache__/
README.md
setup.py*
single_analysis.py*
single_analysis_xrp.py
tess_SPOC/
tess_testlcs/
test.py
Untitled.ipynb
young-cl.xml


In [114]:
f = 'tess_SPOC/4112/0759/hlsp_tess-spoc_tess_phot_0000000141120759-s0006_tess_v1_lc.fits'

---

In [136]:
from astropy.io import fits
from astropy.table import Table

In [117]:
hdul = fits.open(f)

In [119]:
table = hdul[1].data

In [138]:
Table(hdul[1].data)

TIME,TIMECORR,CADENCENO,SAP_FLUX,SAP_FLUX_ERR,SAP_BKG,SAP_BKG_ERR,PDCSAP_FLUX,PDCSAP_FLUX_ERR,QUALITY,PSF_CENTR1,PSF_CENTR1_ERR,PSF_CENTR2,PSF_CENTR2_ERR,MOM_CENTR1,MOM_CENTR1_ERR,MOM_CENTR2,MOM_CENTR2_ERR,POS_CORR1,POS_CORR2
float64,float32,int32,float32,float32,float32,float32,float32,float32,int32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32
1468.2807917680739,-0.0009905346,11559,1659.9418,1.5390537,646.282,0.5556476,nan,nan,8,nan,nan,nan,nan,1828.9194488501248,0.0005690014,306.5861924809694,0.0006710179,nan,nan
1468.3016249890632,-0.0009906383,11560,1663.9329,1.5393502,646.77203,0.5557559,2129.9172,2.0220726,0,nan,nan,nan,nan,1828.920655613533,0.0005675722,306.5854128981614,0.0006696445,0.012474186,0.035992485
1468.3224582102848,-0.0009907418,11561,1660.2052,1.5395511,645.80817,0.55559593,2124.916,2.022337,0,nan,nan,nan,nan,1828.919723140077,0.0005691825,306.586765478536,0.0006711932,0.0117532,0.036631178
1468.3432914315058,-0.0009908453,11562,1660.6776,1.5395408,646.6493,0.5562529,2125.4468,2.022323,0,nan,nan,nan,nan,1828.9217679854578,0.0005689893,306.58291019506106,0.0006708569,0.01275346,0.034869716
1468.3641246528437,-0.0009909486,11563,1660.4423,1.5395685,646.11926,0.5558036,2125.235,2.0223594,0,nan,nan,nan,nan,1828.9202485995495,0.0005695649,306.58404106868204,0.0006707288,0.011914918,0.034875292
1468.3849578744146,-0.0009910518,11564,nan,nan,nan,nan,nan,nan,164,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1468.4057910959855,-0.0009911549,11565,1659.068,1.5386595,646.54694,0.55595493,2125.0662,2.0211656,0,nan,nan,nan,nan,1828.9209337931795,0.0005693579,306.5834979624247,0.0006714196,0.011527848,0.031968445
1468.4266243177897,-0.0009912578,11566,1659.739,1.538699,645.46747,0.5557391,2124.5786,2.0212173,0,nan,nan,nan,nan,1828.9208891997498,0.00056901725,306.5824767182869,0.0006708841,0.012615871,0.031976853
1468.4474575395925,-0.0009913608,11567,1659.9774,1.538291,646.57135,0.55530286,2124.843,2.0206814,0,nan,nan,nan,nan,1828.9206969161014,0.0005686776,306.5812170089842,0.00067067845,0.012100475,0.031573374


---

In [132]:
g = 'kepler/kplr003542116-2012088054726_llc.fits'


In [134]:
hdul_kepler = fits.open(g)

In [139]:
Table(hdul_kepler[1].data)

TIME,TIMECORR,CADENCENO,SAP_FLUX,SAP_FLUX_ERR,SAP_BKG,SAP_BKG_ERR,PDCSAP_FLUX,PDCSAP_FLUX_ERR,SAP_QUALITY,PSF_CENTR1,PSF_CENTR1_ERR,PSF_CENTR2,PSF_CENTR2_ERR,MOM_CENTR1,MOM_CENTR1_ERR,MOM_CENTR2,MOM_CENTR2_ERR,POS_CORR1,POS_CORR2
float64,float32,int32,float32,float32,float32,float32,float32,float32,int32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32
1099.4081068333253,-0.0015526867,48473,1396269.5,30.041786,5616.551,1.5368145,nan,nan,0,nan,nan,nan,nan,861.0025442447447,2.059641e-05,336.396691701358,2.9850946e-05,-0.09032409,0.15713888
1099.428539901186,-0.0015533188,48474,1396251.4,30.042507,5608.0703,1.538455,1409348.9,30.312864,0,nan,nan,nan,nan,861.002934054182,2.0594432e-05,336.3968758175385,2.9862764e-05,-0.08994268,0.15725943
1099.4489728694025,-0.0015539506,48475,1396199.4,30.041105,5613.1265,1.5375682,1409303.9,30.319302,0,nan,nan,nan,nan,861.0024493283292,2.059419e-05,336.39649215583415,2.9856566e-05,-0.090236284,0.15701286
1099.4694059379617,-0.001554582,48476,1396177.0,30.041704,5616.4785,1.5353531,1409300.0,30.301926,0,nan,nan,nan,nan,861.0027064441165,2.059521e-05,336.39681995460546,2.986206e-05,-0.09005718,0.15734409
1099.4898388067595,-0.0015552132,48477,1396156.4,30.04156,5615.6587,1.5374116,1409284.1,30.309134,0,nan,nan,nan,nan,861.002500217406,2.0593814e-05,336.3969062762443,2.9862154e-05,-0.09018305,0.15744616
1099.5102717759073,-0.0015558441,48478,1396219.5,30.041262,5609.3936,1.5381569,1409346.6,30.31892,0,nan,nan,nan,nan,861.0022526579553,2.059295e-05,336.39643476724376,2.9855883e-05,-0.09034959,0.1570332
1099.5307048452814,-0.0015564747,48479,1396254.0,30.042488,5610.4507,1.5383259,1409391.9,30.309227,0,nan,nan,nan,nan,861.002411591037,2.0593698e-05,336.3966378151431,2.986041e-05,-0.09027792,0.15720372
1099.5511378151277,-0.0015571049,48480,1396169.9,30.040709,5620.035,1.5357639,1409291.0,30.324562,0,nan,nan,nan,nan,861.0021663708305,2.0593261e-05,336.3964065724989,2.9857174e-05,-0.09041227,0.15694891
1099.5715706850897,-0.0015577349,48481,1396180.0,30.04126,5615.8667,1.5370452,1409291.1,30.322155,0,nan,nan,nan,nan,861.0023007503851,2.059395e-05,336.3963459754981,2.9859584e-05,-0.09033606,0.15681532
